In [1]:
import time
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from imblearn import pipeline
from lightgbm import LGBMRegressor
from sklearn import preprocessing, compose, metrics
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [5]:
np.random.seed(0)

In [6]:
#importing file
df = pd.read_csv('./data/london_merged.csv')
df.isnull().sum() #no null value
df

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...
17409,2017-01-03 19:00:00,1042,5.0,1.0,81.0,19.0,3.0,0.0,0.0,3.0
17410,2017-01-03 20:00:00,541,5.0,1.0,81.0,21.0,4.0,0.0,0.0,3.0
17411,2017-01-03 21:00:00,337,5.5,1.5,78.5,24.0,4.0,0.0,0.0,3.0
17412,2017-01-03 22:00:00,224,5.5,1.5,76.0,23.0,4.0,0.0,0.0,3.0


In [7]:
df['hour'] = df['timestamp'].apply(lambda row: row.split(':')[0][-2:])

In [8]:
x, y = df.drop(['cnt','timestamp'], axis=1), df['cnt'] #selecting x and y
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [9]:
x_train['cnt'] = y_train 

<ipython-input-9-f6c73911930b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['cnt'] = y_train


In [10]:
def enhancement(train, dev):
    gen_data = train.copy()
    for season in train['season'].unique():
        seasonal_data = gen_data[gen_data['season'] == season]
        hum_std = seasonal_data['hum'].std()
        wind_speed_std = seasonal_data['wind_speed'].std()
        t1_std = seasonal_data['t1'].std()
        t2_std = seasonal_data['t2'].std()

        for row in train[train['season']==season].index:

            if np.random.randint(2)==1:
                gen_data.loc[row, 'hum'] += hum_std/dev
            else:
                gen_data.loc[row, 'hum'] -= hum_std/dev

            if np.random.randint(2)==1:
                gen_data.loc[row, 'wind_speed'] += wind_speed_std/dev
            else:
                gen_data.loc[row, 'wind_speed'] -= wind_speed_std/dev

            if np.random.randint(2)==1:
                gen_data.loc[row, 't1'] += t1_std/dev
            else:
                gen_data.loc[row, 't1'] -= t1_std/dev

            if np.random.randint(2)==1:
                gen_data.loc[row, 't2'] += t2_std/dev
            else:
                gen_data.loc[row, 't2'] -= t2_std/dev

    return gen_data

In [11]:
x_new = enhancement(x_train, 8).sample(x_train.shape[0]//4)
x_train, y_train = pd.concat([x_train, x_new ]).drop('cnt', axis=1), pd.concat([x_train, x_new ])['cnt']